# PySpark Cookbook

### Tomasz Drabas, Denny Lee
#### Version: 0.1
#### Date: 2/28/2018

# Loading the data

In [1]:
import pyspark.sql.functions as func
census_path = '../data/census_income.csv'

census = spark.read.csv(
    census_path
    , header=True
    , inferSchema=True
)

for col, typ in census.dtypes:
    if typ == 'string':
        census = census.withColumn(
            col
            , func.ltrim(func.rtrim(census[col]))
        )
census.count()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,None,pyspark,idle,,,✔


SparkSession available as 'spark'.
32561

In [2]:
census.show()

+---+----------------+------+------------+-------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+-----+
|age|       workclass|fnlwgt|   education|education-num|      marital-status|       occupation| relationship|              race|   sex|capital-gain|capital-loss|hours-per-week|native-country|label|
+---+----------------+------+------------+-------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+-----+
| 39|       State-gov| 77516|   Bachelors|           13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male|        2174|           0|            40| United-States|<=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors|           13|  Married-civ-spouse|  Exec-managerial|      Husband|             White|  Male|           0|           0|            13| United-States|<=50K|
| 38|     

In [3]:
census.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- label: string (nullable = true)

# Exploring data

## Data prep

List of columns to keep

In [4]:
cols_to_keep = census.dtypes

cols_to_keep = (
    ['label','age'
     ,'capital-gain'
     ,'capital-loss'
     ,'hours-per-week'
    ] + [
        e[0] for e in cols_to_keep[:-1] 
        if e[1] == 'string'
    ]
)

cols_to_keep

['label', 'age', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

Get numeric and categorical columns

In [5]:
import pyspark.mllib.stat as st
import numpy as np

census_subset = census.select(cols_to_keep)

cols_num = [
    e[0] for e in census_subset.dtypes 
    if e[1] == 'int'
]
cols_cat = [
    e[0] for e in census_subset.dtypes[1:] 
    if e[1] == 'string'
]
cols_num, cols_cat

(['age', 'capital-gain', 'capital-loss', 'hours-per-week'], ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])

## Numerical data

In [6]:
rdd_num = (
    census_subset
    .select(cols_num)
    .rdd
    .map(lambda row: [e for e in row])
)

stats_num = st.Statistics.colStats(rdd_num)

for col, min_, mean_, max_, var_ in zip(
      cols_num
    , stats_num.min()
    , stats_num.mean()
    , stats_num.max()
    , stats_num.variance()
):
    print('{0}: min->{1:.1f}, mean->{2:.1f}, max->{3:.1f}, stdev->{4:.1f}'
          .format(col, min_, mean_, max_, np.sqrt(var_)))

age: min->17.0, mean->38.6, max->90.0, stdev->13.6
capital-gain: min->0.0, mean->1077.6, max->99999.0, stdev->7385.3
capital-loss: min->0.0, mean->87.3, max->4356.0, stdev->403.0
hours-per-week: min->1.0, mean->40.4, max->99.0, stdev->12.3

## Categorical data

In [7]:
rdd_cat = (
    census_subset
    .select(cols_cat + ['label'])
    .rdd
    .map(lambda row: [e for e in row])
)

results_cat = {}

for i, col in enumerate(cols_cat + ['label']):
    results_cat[col] = (
        rdd_cat
        .groupBy(lambda row: row[i])
        .map(lambda el: (el[0], len(el[1])))
        .collect()
    )

for k in results_cat:
    print(
        k
        , sorted(
            results_cat[k]
            , key=lambda el: el[1]
            , reverse=True)
        , '\n')

sex [('Male', 21790), ('Female', 10771)] 

race [('White', 27816), ('Black', 3124), ('Asian-Pac-Islander', 1039), ('Amer-Indian-Eskimo', 311), ('Other', 271)] 

label [('<=50K', 24720), ('>50K', 7841)] 

native-country [('United-States', 29170), ('Mexico', 643), ('?', 583), ('Philippines', 198), ('Germany', 137), ('Canada', 121), ('Puerto-Rico', 114), ('El-Salvador', 106), ('India', 100), ('Cuba', 95), ('England', 90), ('Jamaica', 81), ('South', 80), ('China', 75), ('Italy', 73), ('Dominican-Republic', 70), ('Vietnam', 67), ('Guatemala', 64), ('Japan', 62), ('Poland', 60), ('Columbia', 59), ('Taiwan', 51), ('Haiti', 44), ('Iran', 43), ('Portugal', 37), ('Nicaragua', 34), ('Peru', 31), ('France', 29), ('Greece', 29), ('Ecuador', 28), ('Ireland', 24), ('Hong', 20), ('Trinadad&Tobago', 19), ('Cambodia', 19), ('Laos', 18), ('Thailand', 18), ('Yugoslavia', 16), ('Outlying-US(Guam-USVI-etc)', 14), ('Hungary', 13), ('Honduras', 13), ('Scotland', 12), ('Holand-Netherlands', 1)] 

marital-statu

## Correlations

In [8]:
correlations = st.Statistics.corr(rdd_num)
correlations

array([[ 1.        ,  0.0776745 ,  0.05777454,  0.06875571],
       [ 0.0776745 ,  1.        , -0.03161506,  0.07840862],
       [ 0.05777454, -0.03161506,  1.        ,  0.05425636],
       [ 0.06875571,  0.07840862,  0.05425636,  1.        ]])

In [9]:
for i, el_i in enumerate(abs(correlations) > 0.05):
    print(cols_num[i])
    
    for j, el_j in enumerate(el_i):
        if el_j and j != i:
            print(
                '    '
                , cols_num[j]
                , correlations[i][j]
            )
            
    print()

age
     capital-gain 0.077674498166
     capital-loss 0.057774539479
     hours-per-week 0.0687557075095

capital-gain
     age 0.077674498166
     hours-per-week 0.0784086153901

capital-loss
     age 0.057774539479
     hours-per-week 0.0542563622727

hours-per-week
     age 0.0687557075095
     capital-gain 0.0784086153901
     capital-loss 0.0542563622727

# Statistical testing

In [10]:
import pyspark.mllib.linalg as ln

census_occupation = (
    census
    .groupby('occupation')
    .pivot('label')
    .count()
)

census_occupation_coll = (
    census_occupation
    .rdd
    .map(lambda row: (row[1:]))
    .flatMap(lambda row: row)
    .collect()
)

len_row = len(census_occupation.collect())
dense_mat = ln.DenseMatrix(
    len_row
    , 2
    , census_occupation_coll
    , True
)

chi_sq = st.Statistics.chiSqTest(dense_mat)

print(chi_sq.pValue)
print(chi_sq.nullHypothesis)

0.0
the occurrence of the outcomes is statistically independent.

In [11]:
dense_mat.toArray()

array([[  2.66700000e+03,   9.83000000e+02],
       [  2.09800000e+03,   1.96800000e+03],
       [  2.28100000e+03,   1.85900000e+03],
       [  1.28400000e+03,   8.60000000e+01],
       [  8.79000000e+02,   1.15000000e+02],
       [  3.17000000e+03,   9.29000000e+02],
       [  1.27700000e+03,   3.20000000e+02],
       [  1.48000000e+02,   1.00000000e+00],
       [  4.38000000e+02,   2.11000000e+02],
       [  3.15800000e+03,   1.37000000e+02],
       [  6.45000000e+02,   2.83000000e+02],
       [  1.75200000e+03,   2.50000000e+02],
       [  8.00000000e+00,   1.00000000e+00],
       [  1.65200000e+03,   1.91000000e+02],
       [  3.26300000e+03,   5.07000000e+02]])

# Transforming the data

Number of distinct values

In [12]:
len_ftrs = []

for col in cols_cat:
    (
        len_ftrs
        .append(
            (col
             , census
                 .select(col)
                 .distinct()
                 .count()
            )
        )
    )
    
len_ftrs = dict(len_ftrs)
len_ftrs

{'sex': 2, 'race': 5, 'native-country': 42, 'marital-status': 7, 'workclass': 9, 'education': 16, 'occupation': 15, 'relationship': 6}

Using hashing trick

In [13]:
import pyspark.mllib.feature as feat

final_data = (
    census
    .select(cols_to_keep)
    .rdd
    .map(lambda row: [
        list(
            feat.HashingTF(int(len_ftrs[col] / 2.0))
            .transform(row[i])
            .toArray()
        ) if i >= 5
        else [row[i]] 
        for i, col in enumerate(cols_to_keep)]
    )
)

final_data.take(3)

[[['<=50K'], [39], [2174], [0], [40], [1.0, 2.0, 1.0, 5.0], [3.0, 3.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0], [2.0, 3.0, 8.0], [0.0, 3.0, 3.0, 1.0, 4.0, 1.0, 0.0], [5.0, 5.0, 3.0], [3.0, 2.0], [4.0], [1.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.0]], [['<=50K'], [50], [0], [0], [13], [4.0, 3.0, 1.0, 8.0], [3.0, 3.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0], [5.0, 5.0, 8.0], [0.0, 1.0, 2.0, 2.0, 8.0, 1.0, 1.0], [4.0, 2.0, 1.0], [3.0, 2.0], [4.0], [1.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.0]], [['<=50K'], [38], [0], [0], [40], [2.0, 2.0, 0.0, 3.0], [2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0], [3.0, 2.0, 3.0], [2.0, 3.0, 1.0, 3.0, 7.0, 0.0, 1.0], [5.0, 5.0, 3.0], [3.0, 2.0], [4.0], [1.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.0]]]

Encode label

In [14]:
def labelEncode(label):
    return [int(label[0] == '>50K')]

final_data = (
    final_data
    .map(lambda row: labelEncode(row[0]) 
         + [item 
            for sublist in row[1:] 
            for item in sublist]
        )
)

final_data.take(3)

[[0, 39, 2174, 0, 40, 1.0, 2.0, 1.0, 5.0, 3.0, 3.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 2.0, 3.0, 8.0, 0.0, 3.0, 3.0, 1.0, 4.0, 1.0, 0.0, 5.0, 5.0, 3.0, 3.0, 2.0, 4.0, 1.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.0], [0, 50, 0, 0, 13, 4.0, 3.0, 1.0, 8.0, 3.0, 3.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 5.0, 5.0, 8.0, 0.0, 1.0, 2.0, 2.0, 8.0, 1.0, 1.0, 4.0, 2.0, 1.0, 3.0, 2.0, 4.0, 1.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.0], [0, 38, 0, 0, 40, 2.0, 2.0, 0.0, 3.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 3.0, 2.0, 3.0, 2.0, 3.0, 1.0, 3.0, 7.0, 0.0, 1.0, 5.0, 5.0, 3.0, 3.0, 2.0, 4.0, 1.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.0]]

# Creating an RDD for training

In [15]:
import pyspark.mllib.regression as reg

final_data_income = (
    final_data
    .map(lambda row: reg.LabeledPoint(
        row[0]
        , ln.Vectors.dense(row[1:]))
        )
)
final_data_income.take(2)

[LabeledPoint(0.0, [39.0,2174.0,0.0,40.0,1.0,2.0,1.0,5.0,3.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,3.0,8.0,0.0,3.0,3.0,1.0,4.0,1.0,0.0,5.0,5.0,3.0,3.0,2.0,4.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,0.0]), LabeledPoint(0.0, [50.0,0.0,0.0,13.0,4.0,3.0,1.0,8.0,3.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,5.0,8.0,0.0,1.0,2.0,2.0,8.0,1.0,1.0,4.0,2.0,1.0,3.0,2.0,4.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,0.0])]

In [16]:
final_data_hours = (
    final_data
    .map(lambda row: reg.LabeledPoint(
        row[4]
        , ln.Vectors.dense(row[0:4] + row[5:]))
        )
)
final_data_hours.take(2)

[LabeledPoint(40.0, [0.0,39.0,2174.0,0.0,1.0,2.0,1.0,5.0,3.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,3.0,8.0,0.0,3.0,3.0,1.0,4.0,1.0,0.0,5.0,5.0,3.0,3.0,2.0,4.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,0.0]), LabeledPoint(13.0, [0.0,50.0,0.0,0.0,4.0,3.0,1.0,8.0,3.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,5.0,8.0,0.0,1.0,2.0,2.0,8.0,1.0,1.0,4.0,2.0,1.0,3.0,2.0,4.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,0.0])]

# Splitting into training and testing

In [17]:
(
    final_data_income_train
    , final_data_income_test
) = (
    final_data_income.randomSplit([0.7, 0.3])
)

In [18]:
(
    final_data_hours_train
    , final_data_hours_test
) = (
    final_data_hours.randomSplit([0.7, 0.3])
)

# Predicting hours of work for census respondents

Linear regression (benchmark)

In [19]:
workhours_model_lm = reg.LinearRegressionWithSGD.train(
    final_data_hours_train
    , iterations = 10
)

/opt/spark/python/lib/pyspark.zip/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.

In [20]:
small_sample_hours = sc.parallelize(final_data_hours_test.take(10))

for t,p in zip(
    small_sample_hours
        .map(lambda row: row.label)
        .collect()
    , workhours_model_lm.predict(
        small_sample_hours
            .map(lambda row: row.features)
    ).collect()):
    print(t,p)

40.0 -5.75389450988e+69
16.0 -7.41875981037e+69
45.0 -7.88395163906e+69
50.0 -2.16395670855e+75
50.0 -4.98447372989e+69
40.0 -6.19604774416e+69
45.0 -5.39661643836e+69
35.0 -4.15537088245e+69
40.0 -5.0276020275e+69
40.0 -8.55190824921e+69

# Forecasting income levels of census respondents

Logistic regression

In [21]:
import pyspark.mllib.classification as cl

income_model_lr = cl.LogisticRegressionWithSGD.train(
    final_data_income_train
    , iterations=10
)

/opt/spark/python/lib/pyspark.zip/pyspark/mllib/classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.

In [22]:
small_sample_income = sc.parallelize(final_data_income_test.take(10))

for t,p in zip(
    small_sample_income
        .map(lambda row: row.label)
        .collect()
    , income_model_lr.predict(
        small_sample_income
            .map(lambda row: row.features)
    ).collect()):
    print(t,p)

0.0 1
0.0 1
0.0 1
1.0 1
1.0 1
0.0 1
0.0 1
1.0 1
0.0 1
0.0 1

In [23]:
income_model_lr.threshold

0.5

In [24]:
income_model_lr.weights

DenseVector([1.7096, 269.308, -7.9421, 1.3769, 0.0074, -0.0321, 0.021, -0.0719, 0.1393, -0.0532, -0.1522, -0.04, 0.0985, 0.0468, -0.3336, 0.009, 0.689, 0.6593, -0.0671, -0.0023, -0.0763, -0.1802, -0.1886, -0.3079, -0.0625, 0.1184, -0.4948, -0.8701, -1.1213, -0.2172, -0.287, -0.7601, -0.0636, -0.0038, -0.0084, -0.192, -0.0023, -0.0618, -0.0033, -0.0015, 0.0035, -0.015, 0.0, -0.1738, -0.0313, -0.006, -0.0161, -0.072, -0.0719, -0.1462, -0.0727, -0.0875, 0.0])

Support Vector Machines

In [25]:
income_model_svm = cl.SVMWithSGD.train(
    final_data_income
    , iterations=100
    , step=0.98
    , miniBatchFraction=1/3.0
)

In [26]:
for t,p in zip(
    small_sample_income
        .map(lambda row: row.label)
        .collect()
    , income_model_svm.predict(
        small_sample_income
            .map(lambda row: row.features)
    ).collect()):
    print(t,p)

0.0 0
0.0 0
0.0 0
1.0 1
1.0 0
0.0 0
0.0 0
1.0 0
0.0 0
0.0 0

In [27]:
income_model_svm.weights

DenseVector([0.2073, 30.1347, 4.1533, 0.388, -0.0469, -0.1314, 0.0327, -0.4033, 0.457, -0.129, -0.4897, -0.152, 0.3058, 0.1524, -1.134, -0.0676, 1.665, 1.7398, -0.3221, -0.063, -0.1754, -0.5714, -0.6371, -0.9501, -0.2668, 0.373, -1.6258, -2.5894, -3.3322, -0.7352, -0.8754, -2.3881, -0.2096, -0.0141, -0.0265, -0.6443, -0.0026, -0.2072, -0.0107, -0.0049, 0.0078, -0.0532, 0.0, -0.5889, -0.1017, -0.0148, -0.0541, -0.2429, -0.2379, -0.4811, -0.2493, -0.2859, 0.0])

# Building clustering models

In [28]:
import pyspark.mllib.clustering as clu

model = clu.KMeans.train(
    final_data.map(lambda row: row[1:])
    , 2
    , maxIterations=10
    , initializationMode='random'
    , seed=666
    , initializationSteps=5
    , epsilon=1e-4
)

In [29]:
import sklearn.metrics as m

predicted = (
    model
        .predict(
            final_data.map(lambda row: row[1:])
        )
)
predicted = predicted.collect()

true = final_data.map(lambda row: row[0]).collect()

print(m.homogeneity_score(true, predicted))
print(m.completeness_score(true, predicted))

0.0126632823359
0.226522171727

# Computing performance statistics

In [30]:
import pyspark.mllib.evaluation as ev

Regression metrics

In [31]:
true_pred_reg = (
    final_data_hours_test
    .map(lambda row: (
         float(workhours_model_lm.predict(row.features))
         , row.label))
)

metrics_lm = ev.RegressionMetrics(true_pred_reg)

In [32]:
print('R^2: ', metrics_lm.r2)
print('Explained Variance: ', metrics_lm.explainedVariance)
print('meanAbsoluteError: ', metrics_lm.meanAbsoluteError)

R^2:  -8.23790361194809e+147
Explained Variance:  1.2821818769638103e+150
meanAbsoluteError:  1.613057724820514e+74

Classification metrics

In [33]:
true_pred_class_lr = (
    final_data_income_test
    .map(lambda row: (
        float(income_model_lr.predict(row.features))
        , row.label))
)

metrics_lr = ev.BinaryClassificationMetrics(true_pred_class_lr)

print('areaUnderPR: ', metrics_lr.areaUnderPR)
print('areaUnderROC: ', metrics_lr.areaUnderPR)

areaUnderPR:  0.5768927693066335
areaUnderROC:  0.5768927693066335

In [34]:
trainErr = (
    true_pred_class_lr
    .filter(lambda lp: lp[0] != lp[1]).count() 
    / float(true_pred_class_lr.count())
)
print("Training Error = " + str(trainErr))

Training Error = 0.7555487368313388

In [35]:
true_pred_class_svm = (
    final_data_income_test
    .map(lambda row: (
        float(income_model_svm.predict(row.features))
        , row.label))
)

metrics_svm = ev.BinaryClassificationMetrics(true_pred_class_svm)

print('areaUnderPR: ', metrics_svm.areaUnderPR)
print('areaUnderROC: ', metrics_svm.areaUnderPR)

areaUnderPR:  0.5764976815370444
areaUnderROC:  0.5764976815370444

In [36]:
trainErr = (
    true_pred_class_svm
    .filter(lambda lp: lp[0] != lp[1]).count() 
    / float(true_pred_class_svm.count())
)

print("Training Error = " + str(trainErr))

Training Error = 0.22634755037332516

Confusion matrix

In [37]:
(
    true_pred_class_svm
    .map(lambda el: ((el), 1))
    .reduceByKey(lambda x,y: x+y)
    .take(4)
)

[((0.0, 1.0), 1250), ((1.0, 0.0), 963), ((0.0, 0.0), 6410), ((1.0, 1.0), 1154)]

In [38]:
(
    true_pred_class_lr
    .map(lambda el: ((el), 1))
    .reduceByKey(lambda x,y: x+y)
    .take(4)
)

[((0.0, 1.0), 249), ((1.0, 0.0), 7138), ((0.0, 0.0), 235), ((1.0, 1.0), 2155)]